# Introduction to Quantum Computing

## Problem setting
Suppose you have a function $f(x)$. This function can be one of four types:
1. $f(x) = 0$
2. $f(x) = 1$
3. $f(x) = x$
4. $f(x) = \overline{x}$

The first two functions are constant ones because they return the same bit whatever input is. While the last two functions outputs a bit depending on input bit. Now problem is stated to determine whether this function constant or not?

How many steps do you need to solve this problem in a classical way? Well, you need exactly two steps to determine the type of function. Quantum computer is able to perform the same task in a single step. How it could be possible? Let's deal with it. 

## Quantum computing properties
Quantum computer has two features which make it potentially more powerfull then classical computers. They are superposition and entaglement.  

## Calculations

$ |01 \rangle \xrightarrow{H} \frac{1}{2} (|0\rangle + |1\rangle) (|0\rangle - |1\rangle) = $

In [14]:
import quant
import numpy as np
from functools import reduce

Given a function $ U_f |xy\rangle= |x\rangle | y \oplus f(x) \rangle $


Goal is to determine the type of $f$.

# Deutsch algorithm
![title](deutsch.png)

## Step 1

Given two inputs: 
- $x_0$ first input in a state $|0\rangle$
- $x_1$ second input in a state $|1\rangle$


In [8]:
x0 = np.array([[1],
               [0]])
x1 = np.array([[0], 
            [1]])

The overall state is $|01\rangle$.

In [9]:
x01 = np.kron(x0, x1)

Create two-qubit Hadamard operator and apply it to $|01\rangle$.

In [10]:
H2 = hadamard_n(2)
x01_H2 = np.dot(H2, x01)

In [11]:
x01_H2

array([[ 0.5],
       [-0.5],
       [ 0.5],
       [-0.5]])

## Step 2
Now we need to express function $f: \{0, 1\} \rightarrow \{0, 1\}$ as a valid unitary matrix.

Apply $U_f$ to qubits.

In [7]:
f1 = lambda x: 0
f2 = lambda x: 1
f3 = lambda x: x
f4 = lambda x: 1 - x

In [16]:
def create_operator_matrix_deutsch(qubit_number, func):
    # qubit number only two
    states = 2 ** qubit_number
    res = np.empty([states, states])
    
    for i in range(states):
        state = format(i, '0' + str(qubit_number) + 'b')
        x0 = state[0]
        x1 = str(int(state[1]) ^ func(int(state[0])))
        column = quant.string_to_state(x0+x1)
        res[:,i] = column[:, 0]
    
    return res

In [17]:
U_f = create_operator_matrix_deutsch(2, f1)

In [18]:
X_f = np.dot(U_f, x01_H2)

## Step 3

Apply Hadamard gate to the first qubit.

In [19]:
HI = np.kron(hadamard_n(1), np.identity(2))
final_state = np.dot(HI, X_f)

In [20]:
final_state

array([[ 0.70710678],
       [-0.70710678],
       [ 0.        ],
       [ 0.        ]])

## Measuring the final state 
The probability of qubit $i$ being in a state 0 is given by 
$$Tr [| \psi \rangle \langle \psi | P^{i}_0 ]$$

In [22]:
rho = np.dot(final_state, final_state.T)
P0 = np.dot(quant.ZERO, quant.ZERO.T)
P00 = np.kron(P0, np.eye(2))

prob = np.trace(np.dot(rho, P00))
print(prob)

0.9999999999999996


## Auxiliary

In [ ]:
class QuantumState:
    def __init__(self, alpha: 'complex', beta: 'complex'):
        self.state = np.empty([alpha, beta])

class ZeroState:
    def __init__(self):
        self.state = np.array([1+0j, 0+0j])

class OneState:
    def __init__(self):
        self.state = np.array([0+0j, 0+1j])
        

class QuantumRegister:
    def __init__(self, n: int):
        #let's start with zero
        self.n_qubits = n
        self.n_states = 1 << n
        self.vector = None
        
        self.qubits_states_sep = np.empty(n, 2)
        self.register_state = None
        
        for i in range(len(self.qubits_states)):
            self.qubits_states_sep[i] = np.array([1+0j, 0+0j])
        
        self.full_state = self.get_full_state()
    
    def get_full_state(self, qubits_states_sep):
        from functools import reduce         
        return reduce(np.kron, qubits_states_sep)
    
    @property
    def get_n(self):
        return self.n
        
        

def hadamard(x, Q):
	codomain = []
	for y in range(Q):
		amplitude = complex(pow(-1.0, bitCount(x & y) & 1))
		codomain.append(Mapping(y, amplitude))

	return  codomain

def hadamard(x, Q):
    """
    :type x: vector state to be Hadamardized
    :type Q: 2^n
    """
    codomain = []
    for y in range(Q):
        amplitude = complex(pow(-1., bit_count(x & y) & 1))
        codomain.append(Mapping(y, amplitude))
    return codomain

def hadamard(x, y_i):
    """
    :type x: vector state to be Hadamardized
    :type Q: 2^n
    """
    return np.complex(pow(-1., bit_count(x & y_i) & 1))